In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Functions")
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/13 16:30:37 INFO SparkContext: Running Spark version 3.5.5
25/03/13 16:30:37 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/13 16:30:37 INFO SparkContext: Java version 11.0.26
25/03/13 16:30:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


spark: SparkSession = org.apache.spark.sql.SparkSession@3b80e7ff
import spark.implicits._

In [4]:
val data = Seq(
    ("Jean-Georges", "Perrin", "NC", 1, 300, 1551903533),
    ("Jean-Georges", "Perrin", "NC", 2, 120, 1551903567),
    ("Jean-Georges", "Perrin", "CA" ,4, 75, 1551903599),
    ("Holden", "Karau", "CA" , 6, 37, 1551904299),
    ("Ginni", "Rometty", "NY", 7, 91, 1551916792),
    ("Holden", "Karau", "CA", 4, 153, 1552876129)
).toDF("firstName", "lastName", "state", "quantity", "revenue", "timestamp")

data.show()

+------------+--------+-----+--------+-------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|
+------------+--------+-----+--------+-------+----------+
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|
|      Holden|   Karau|   CA|       6|     37|1551904299|
|       Ginni| Rometty|   NY|       7|     91|1551916792|
|      Holden|   Karau|   CA|       4|    153|1552876129|
+------------+--------+-----+--------+-------+----------+



data: DataFrame = [firstName: string, lastName: string ... 4 more fields]

In [5]:
data.printSchema()

root
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- state: string (nullable = true)
 |-- quantity: integer (nullable = false)
 |-- revenue: integer (nullable = false)
 |-- timestamp: integer (nullable = false)



## Array Functions

In [6]:
val datak2 = data
                .withColumn("k2v", array($"firstName", $"lastName", $"state"))
                .withColumn("k2n", array(lit("FirstName"), lit("LastName"), lit("State")))

datak2.show(false)

+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2v                       |k2n                         |
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[Jean-Georges, Perrin, CA]|[FirstName, LastName, State]|
|Holden      |Karau   |CA   |6       |37     |1551904299|[Holden, Karau, CA]       |[FirstName, LastName, State]|
|Ginni       |Rometty |NY   |7       |91     |1551916792|[Ginni, Rometty, NY]      |[FirstName, LastName, State]|
|Holden      |Karau   |CA   |4       |153    |1552876129|[Holden, Karau, CA]       |[Fir

datak2: DataFrame = [firstName: string, lastName: string ... 6 more fields]

In [7]:
datak2.where(array_contains($"k2v", "CA")).show(false)

+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2v                       |k2n                         |
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[Jean-Georges, Perrin, CA]|[FirstName, LastName, State]|
|Holden      |Karau   |CA   |6       |37     |1551904299|[Holden, Karau, CA]       |[FirstName, LastName, State]|
|Holden      |Karau   |CA   |4       |153    |1552876129|[Holden, Karau, CA]       |[FirstName, LastName, State]|
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+



## Map Functions

In [8]:
val datak2m = data.withColumn("k2m", map(lit("FirstName"), $"firstName", lit("LastName"), $"lastName", lit("State"), $"state"))

datak2m.show(false)

+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2m                                                         |
+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|{FirstName -> Jean-Georges, LastName -> Perrin, State -> CA}|
|Holden      |Karau   |CA   |6       |37     |1551904299|{FirstName -> Holden, LastName -> Karau, State -> CA}       |
|Ginni       |Rometty |NY   |7       |91     |1551916792|{FirstName -> Ginni, LastName -> Rometty, State -> NY}      |
|Holden      |Karau   |CA   |4       |153    |15

datak2m: DataFrame = [firstName: string, lastName: string ... 5 more fields]

In [9]:
val datak2ma = datak2.withColumn("k2m", map_from_arrays($"k2n", $"k2v"))

datak2ma.show(false)

+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+------------------------------------------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2v                       |k2n                         |k2m                                                         |
+------------+--------+-----+--------+-------+----------+--------------------------+----------------------------+------------------------------------------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[Jean-Georges, Perrin, NC]|[FirstName, LastName, State]|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[Jean-Georges, Perrin, CA]|[FirstName, LastName, State]|{FirstName -

datak2ma: DataFrame = [firstName: string, lastName: string ... 7 more fields]

## Date and Timestamp Functions

In [10]:
val datat = data
                .withColumn("recordTimestamp", to_timestamp($"timestamp"))
                .withColumn("current", current_timestamp())

datat.show(false)

+------------+--------+-----+--------+-------+----------+-------------------+--------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |recordTimestamp    |current                   |
+------------+--------+-----+--------+-------+----------+-------------------+--------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|2019-03-06 23:18:53|2025-03-13 16:30:44.995342|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|2019-03-06 23:19:27|2025-03-13 16:30:44.995342|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|2019-03-06 23:19:59|2025-03-13 16:30:44.995342|
|Holden      |Karau   |CA   |6       |37     |1551904299|2019-03-06 23:31:39|2025-03-13 16:30:44.995342|
|Ginni       |Rometty |NY   |7       |91     |1551916792|2019-03-07 02:59:52|2025-03-13 16:30:44.995342|
|Holden      |Karau   |CA   |4       |153    |1552876129|2019-03-18 05:28:49|2025-03-13 16:30:44.995342|
+------------+--------+-----+--------+-------+---------

datat: DataFrame = [firstName: string, lastName: string ... 6 more fields]

In [11]:
datat.printSchema

root
 |-- firstName: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- state: string (nullable = true)
 |-- quantity: integer (nullable = false)
 |-- revenue: integer (nullable = false)
 |-- timestamp: integer (nullable = false)
 |-- recordTimestamp: timestamp (nullable = false)
 |-- current: timestamp (nullable = false)



In [12]:
datat.select($"recordTimestamp", $"current", datediff($"current", $"recordTimestamp")).show(false)

+-------------------+--------------------------+----------------------------------+
|recordTimestamp    |current                   |datediff(current, recordTimestamp)|
+-------------------+--------------------------+----------------------------------+
|2019-03-06 23:18:53|2025-03-13 16:30:45.607223|2199                              |
|2019-03-06 23:19:27|2025-03-13 16:30:45.607223|2199                              |
|2019-03-06 23:19:59|2025-03-13 16:30:45.607223|2199                              |
|2019-03-06 23:31:39|2025-03-13 16:30:45.607223|2199                              |
|2019-03-07 02:59:52|2025-03-13 16:30:45.607223|2198                              |
|2019-03-18 05:28:49|2025-03-13 16:30:45.607223|2187                              |
+-------------------+--------------------------+----------------------------------+



## JSON Functions

In [13]:
val datak2j = datak2m.withColumn("k2j", to_json($"k2m"))

datak2j.show(false)

+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+-------------------------------------------------------------+
|firstName   |lastName|state|quantity|revenue|timestamp |k2m                                                         |k2j                                                          |
+------------+--------+-----+--------+-------+----------+------------------------------------------------------------+-------------------------------------------------------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|{"FirstName":"Jean-Georges","LastName":"Perrin","State":"NC"}|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|{FirstName -> Jean-Georges, LastName -> Perrin, State -> NC}|{"FirstName":"Jean-Georges","LastName":"Perrin","State":"NC"}|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|{FirstName -> Jean-Georges, LastName -

datak2j: DataFrame = [firstName: string, lastName: string ... 6 more fields]

## Generator Functions

In [14]:
val data2 = data
                .withColumn("a1", array(lit(1), lit(2), lit(3), lit(4), lit(5)))
                .withColumn("a2", lit((1 to 5).toArray))

data2.show(false)

+------------+--------+-----+--------+-------+----------+---------------+---------------+
|firstName   |lastName|state|quantity|revenue|timestamp |a1             |a2             |
+------------+--------+-----+--------+-------+----------+---------------+---------------+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Jean-Georges|Perrin  |CA   |4       |75     |1551903599|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Holden      |Karau   |CA   |6       |37     |1551904299|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Ginni       |Rometty |NY   |7       |91     |1551916792|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
|Holden      |Karau   |CA   |4       |153    |1552876129|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|
+------------+--------+-----+--------+-------+----------+---------------+---------------+



data2: DataFrame = [firstName: string, lastName: string ... 6 more fields]

In [15]:
data2.withColumn("dummy", explode($"a1")).show(false)

+------------+--------+-----+--------+-------+----------+---------------+---------------+-----+
|firstName   |lastName|state|quantity|revenue|timestamp |a1             |a2             |dummy|
+------------+--------+-----+--------+-------+----------+---------------+---------------+-----+
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|1    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|2    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|3    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|4    |
|Jean-Georges|Perrin  |NC   |1       |300    |1551903533|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|5    |
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|1    |
|Jean-Georges|Perrin  |NC   |2       |120    |1551903567|[1, 2, 3, 4, 5]|[1, 2, 3, 4, 5]|2    |
|Jean-Georges|Perrin  |NC   |2       |12

## Window Functions

In [16]:
import org.apache.spark.sql.expressions.Window

val windowSpec = Window.partitionBy("firstName", "lastName").orderBy("state")

import org.apache.spark.sql.expressions.Window
windowSpec: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@23981d31

In [17]:
val dataw = data.withColumn("row_number", row_number().over(windowSpec))

dataw.orderBy("firstName", "lastName", "state").show()

+------------+--------+-----+--------+-------+----------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_number|
+------------+--------+-----+--------+-------+----------+----------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|         1|
|      Holden|   Karau|   CA|       6|     37|1551904299|         1|
|      Holden|   Karau|   CA|       4|    153|1552876129|         2|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|         1|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|         2|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|         3|
+------------+--------+-----+--------+-------+----------+----------+



dataw: DataFrame = [firstName: string, lastName: string ... 5 more fields]

In [18]:
dataw.where($"row_number" === 1).show()

+------------+--------+-----+--------+-------+----------+----------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_number|
+------------+--------+-----+--------+-------+----------+----------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|         1|
|      Holden|   Karau|   CA|       6|     37|1551904299|         1|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|         1|
+------------+--------+-----+--------+-------+----------+----------+



In [19]:
dataw.withColumn("revenue2", nth_value($"revenue", 1).over(windowSpec)).show()

+------------+--------+-----+--------+-------+----------+----------+--------+
|   firstName|lastName|state|quantity|revenue| timestamp|row_number|revenue2|
+------------+--------+-----+--------+-------+----------+----------+--------+
|       Ginni| Rometty|   NY|       7|     91|1551916792|         1|      91|
|      Holden|   Karau|   CA|       6|     37|1551904299|         1|      37|
|      Holden|   Karau|   CA|       4|    153|1552876129|         2|      37|
|Jean-Georges|  Perrin|   CA|       4|     75|1551903599|         1|      75|
|Jean-Georges|  Perrin|   NC|       1|    300|1551903533|         2|      75|
|Jean-Georges|  Perrin|   NC|       2|    120|1551903567|         3|      75|
+------------+--------+-----+--------+-------+----------+----------+--------+

